# Classifying visibility

In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split


In [2]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
import numpy as np

base_dataset_path = "/kaggle/input/gamedataset/Dataset"

input_height = 128
input_width = 128

image_shape = (input_height, input_width)

images = []
status = []
for i, game_folder in enumerate(os.listdir(base_dataset_path)):
    game_path = os.path.join(base_dataset_path, game_folder)
    if(game_folder != "Readme.docx"):
        for clip_folder in os.listdir(game_path):
            clip_path = os.path.join(game_path, clip_folder)
            labels = pd.read_csv(os.path.join(clip_path, 'Label.csv'))

            if(len(labels) < 200):
                for _, row in labels.iterrows():
                    if row['visibility'] > 0:
                        image_path = os.path.join(clip_path, row['file name'])
                        image = cv2.resize(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE), (input_height, input_width))
                        image = image.astype(np.float32)

                        images.append(image)
                        status.append(row['status'])
                        

In [ ]:
training_data = {'Path': images, 'Status': status}
df_training = pd.DataFrame(training_data)
df_training.head()

In [ ]:
# Define the dataset class
class GameDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        return self.dataset.loc[index, 'Path'], self.dataset.loc[index, 'Status']

    def __len__(self):
        return len(self.dataset)


In [ ]:
dataset = GameDataset(df_training)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
class BallTrackingCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # [227x227x3] INPUT

        self.conv1 = nn.Conv2d(1, 96, kernel_size=(11, 11), stride=4, padding=0)  # [55x55x96] CONV1: 96 11x11 filters at stride 4, pad 0
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride = 2) # [27x27x96] MAX POOL1: 3x3 filters at stride 2
        self.norm1 = nn.BatchNorm2d(96)  # [27x27x96] NORM1: Normalization layer

        self.conv2 = nn.Conv2d(96, 256, kernel_size=(5, 5), stride=1, padding=2) # [27x27x256] CONV2: 256 5x5 filters at stride 1, pad 2
        self.drop1 = nn.Dropout(0.3)

        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride = 2) # [13x13x256] MAX POOL2: 3x3 filters at stride 2
        self.norm2 = nn.BatchNorm2d(256) # [13x13x256] NORM2: Normalization layer

        self.conv3 = nn.Conv2d(256, 384, kernel_size=(3, 3), stride=1, padding=1)  # [13x13x384] CONV3: 384 3x3 filters at stride 1, pad 1
        self.norm3 = nn.BatchNorm2d(384)

        self.conv4 = nn.Conv2d(384, 384, kernel_size=(3, 3), stride=1, padding=1)  # [13x13x384] CONV4: 384 3x3 filters at stride 1, pad 1
        self.norm4 = nn.BatchNorm2d(384)

        self.conv5 = nn.Conv2d(384, 256, kernel_size=(3, 3), stride=1, padding=1)  # [13x13x256] CONV5: 256 3x3 filters at stride 1, pad 1
        self.norm5 = nn.BatchNorm2d(256)

        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride = 2)  # [6x6x256] MAX POOL3: 3x3 filters at stride 2

        self.relu = nn.ReLU()
        self.flat = nn.Flatten()
        self.drop2 = nn.Dropout(0.5)

        #self.fc6 = nn.Linear(9216, 4096)     
        self.fc6 = nn.Linear(1024, 512)     
        
        #self.fc7 = nn.Linear(4096, 512)      
        self.fc8 = nn.Linear(512, 4)        # output classes are 4

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x))) 
        x = self.drop1(x)
        x = self.pool2(self.relu(self.conv2(x))) 
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool3(self.relu(self.conv5(x))) 
        x = self.drop2(self.flat(x))
        x = self.relu(self.fc6(x))
        x = self.drop2(x)
        #x = self.relu(self.fc7(x))
        x = self.fc8(x)
        return x


In [ ]:
def train_model(model, optimizer, loss_fn, n_epochs=50, early_stopping = False, plot_metrics = False):

    for epoch in range(n_epochs):

        train_loss = []
        correct_train_pred = 0
        
        for inputs, labels in train_loader:
            inputs =  inputs[:, np.newaxis, :, :].to(device)
            labels = labels.to(device)
            
            y_pred = model(inputs)

            pred_proba = nn.Softmax(dim=1)(y_pred)
            prob_index = torch.argmax(pred_proba, dim=1)
            prob_index = prob_index.float()

            loss = loss_fn(pred_proba, labels.long())  # Convert labels to long type
            train_loss.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            correct_train_pred += (prob_index == labels.data).sum().item()

        train_acc = round((correct_train_pred / (len(train_loader) * batch_size)) * 100, 4)
        train_loss = round(sum(train_loss)/(len(train_loader) * batch_size), 4)

        val_loss = []
        correct_val_pred = 0
        for inputs, labels in val_loader:
            inputs = inputs =  inputs[:, np.newaxis, :, :].to(device)
            labels = labels.to(device)

            y_pred = model(inputs)

            pred_proba = nn.Softmax(dim=1)(y_pred)
            prob_index = torch.argmax(pred_proba, dim=1)

            prob_index = prob_index.float()
            
            loss = loss_fn(pred_proba, labels.long())  # Convert labels to long type
            val_loss.append(loss.item())

            correct_val_pred += (prob_index == labels.data).sum().item()

        val_acc = round((correct_val_pred / (len(val_loader) * batch_size)) * 100, 4)
        val_loss = round(sum(val_loss)/(len(val_loader) * batch_size), 4)

        print(f'epoch {epoch + 1}, training accuracy: {train_acc}, training loss: {train_loss}, validation accuracy: {val_acc}, validation loss: {val_loss}')
            
    return model

model = BallTrackingCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #, weight_decay = 0.0005)
n_epochs = 10

model = train_model(model, optimizer, loss_fn, n_epochs)


# Regressing x and y coordinates 

In [31]:
import numpy as np
import cv2
import os
import pandas as pd
import numpy as np

base_dataset_path = "/kaggle/input/gamedataset/Dataset"

input_height = 72
input_width = 128

image_shape = (input_height, input_width)
clip_counts = {'game1': 13, 'game2': 8, 'game3': 9}

heatmaps = []
x_axis = []
y_axis = []
for i, game_folder in enumerate(os.listdir(base_dataset_path)):
    game_path = os.path.join(base_dataset_path, game_folder)
    if(game_folder != "Readme.docx"):
        for clip_folder in os.listdir(game_path):
            clip_path = os.path.join(game_path, clip_folder)
            labels = pd.read_csv(os.path.join(clip_path, 'Label.csv'))
            
            #if(len(labels) < 200):

            for _, row in labels.iterrows():
                if row['visibility'] > 0:
                    image_path = os.path.join(clip_path, row['file name'])
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    original_shape = image.shape

                    image = cv2.resize(image, (input_height, input_width))
                    image = image.astype(np.float32)
                    heatmaps.append(image)

                    # normalise the coordinate values by dividing them by the input shape
                    x_axis.append(np.array([round(row['x-coordinate'] / (10 * input_width),4), round(row['y-coordinate'] / (10 * input_height), 4)]).astype(np.float32))




In [33]:
training_data = {'Image': heatmaps, 'coord': x_axis}
df_training = pd.DataFrame(training_data)
df_training.head()

,Image,coord
0,"[[164.0, 170.0, 172.0, 172.0, 172.0, 176.0, 17...","[0.4492, 0.0847]"
1,"[[164.0, 170.0, 172.0, 172.0, 172.0, 176.0, 17...","[0.4492, 0.0708]"
2,"[[164.0, 171.0, 172.0, 172.0, 172.0, 176.0, 17...","[0.45, 0.0639]"
3,"[[164.0, 171.0, 172.0, 172.0, 172.0, 176.0, 17...","[0.45, 0.0528]"
4,"[[164.0, 171.0, 172.0, 172.0, 172.0, 176.0, 17...","[0.45, 0.0458]"


In [34]:
# Define the dataset class
class GameDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        return self.dataset.loc[index, 'Image'], self.dataset.loc[index, 'coord']

    def __len__(self):
        return len(self.dataset)


In [41]:
dataset = GameDataset(df_training)

train_size = int(0.7 * len(dataset))
val_size = (len(dataset) - train_size) // 2 
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, val_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class customTrackNet(nn.Module):
    def __init__(self, input_height, input_width):
        super(customTrackNet, self).__init__()
        self.input_height = input_height
        self.input_width = input_width

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * (self.input_height // 16) * (self.input_width // 16), 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)

        # Output layer
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        # Convolutional layers
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4(x))
        x = self.pool4(x)

        # Flatten the output
        x = x.view(-1, 256 * (self.input_height // 16) * (self.input_width // 16))

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        # Output layer
        x = F.softmax(self.fc3(x), dim=1)
        return x

# Create an instance of the TrackNet model
model = customTrackNet(input_height, input_width)


In [37]:
def train_model(model, optimizer, loss_fn, n_epochs=50, early_stopping = False, plot_metrics = False):
    for epoch in range(n_epochs):
        train_loss = []
        correct_train_pred = 0
        for inputs, labels in train_loader:

            inputs = inputs[:, np.newaxis, :, :].to(torch.float).to(device)
            labels = labels.to(device)

            y_pred = model(inputs)
            loss = loss_fn(y_pred, labels)
            train_loss.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss = round(sum(train_loss)/(len(train_loader) * batch_size), 4)

        val_loss = []
        correct_val_pred = 0
        for inputs, labels in val_loader:

            inputs = inputs[:, np.newaxis, :, :].to(torch.float).to(device)
            labels = labels.to(device)

            y_pred = model(inputs)
            loss = loss_fn(y_pred, labels)
            val_loss.append(loss.item())

        val_loss = round(sum(val_loss)/(len(val_loader) * batch_size), 4)
        print(f'epoch {epoch + 1}, training loss: {train_loss}, validation loss: {val_loss}')

    return model

tracknet_model = customTrackNet(input_height, input_width).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(tracknet_model.parameters(), lr=0.0001)
n_epochs = 20

# Calculate the loss
tracknet_model = train_model(tracknet_model, optimizer, loss_fn, n_epochs)


epoch 1, training loss: 0.001, validation loss: 0.0009
epoch 2, training loss: 0.0009, validation loss: 0.0009
epoch 3, training loss: 0.0009, validation loss: 0.0009
epoch 4, training loss: 0.0008, validation loss: 0.0008
epoch 5, training loss: 0.0008, validation loss: 0.0008
epoch 6, training loss: 0.0007, validation loss: 0.0007
epoch 7, training loss: 0.0007, validation loss: 0.0007
epoch 8, training loss: 0.0007, validation loss: 0.0007
epoch 9, training loss: 0.0006, validation loss: 0.0007
epoch 10, training loss: 0.0006, validation loss: 0.0007
epoch 11, training loss: 0.0006, validation loss: 0.0007
epoch 12, training loss: 0.0006, validation loss: 0.0006
epoch 13, training loss: 0.0006, validation loss: 0.0006
epoch 14, training loss: 0.0006, validation loss: 0.0006
epoch 15, training loss: 0.0006, validation loss: 0.0006
epoch 16, training loss: 0.0006, validation loss: 0.0006
epoch 17, training loss: 0.0006, validation loss: 0.0006
epoch 18, training loss: 0.0006, validati

In [91]:
df_pred = df_training[-val_size:].reset_index(drop=True)
df_pred

,Image,coord
0,"[[56.0, 48.0, 133.0, 3.0, 31.0, 27.0, 36.0, 39...","[0.5312, 0.3708]"
1,"[[56.0, 48.0, 133.0, 3.0, 31.0, 27.0, 36.0, 39...","[0.518, 0.35]"
2,"[[56.0, 48.0, 133.0, 3.0, 31.0, 27.0, 36.0, 39...","[0.5188, 0.3528]"
3,"[[57.0, 48.0, 133.0, 3.0, 31.0, 27.0, 36.0, 39...","[0.5078, 0.3403]"
4,"[[57.0, 48.0, 133.0, 3.0, 31.0, 27.0, 36.0, 39...","[0.4984, 0.3347]"
...,...,...
2861,"[[84.0, 94.0, 82.0, 85.0, 71.0, 87.0, 72.0, 81...","[0.1859, 0.6639]"
2862,"[[84.0, 94.0, 82.0, 85.0, 71.0, 87.0, 72.0, 81...","[0.1734, 0.6833]"
2863,"[[84.0, 94.0, 82.0, 85.0, 71.0, 87.0, 72.0, 81...","[0.1602, 0.7111]"
2864,"[[84.0, 94.0, 82.0, 85.0, 71.0, 87.0, 72.0, 81...","[0.1484, 0.7361]"


In [ ]:
listPred = []
df_pred.loc[:, 'x'] = 0
df_pred.loc[:, 'y'] = 0

count = 0
for i, values in enumerate(test_loader):
    inputs, _ = values
    inputs = inputs[:, np.newaxis, :, :].to(torch.float).to(device)
    y_pred = tracknet_model(inputs).cpu().detach().numpy()
    size = len(y_pred)
    
    # 0 - x-coordinate, 1 - y-coordinate
    for j in range(0:20):
        print(y_pred[j, 0]  * input_width , df_pred.loc[j, 'coord'][0] *  input_width, y_pred[j, 1] * input_height , df_pred.loc[j, 'coord'][1] * input_height)
    